# Part 2: Rediscover the Higgs boson

In [1]:
import uproot3 as uproot
import uproot_methods.classes.TLorentzVector as LVepm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

# Step 1: Access a the data file and convert into a 'pandas dataframe'.


In [4]:
eventsData = uproot.open("data_Skim_GamGam.root")["mini"]
df = eventsData.pandas.df(["photon_pt", "photon_eta", "photon_phi", "photon_E",  "diphoton_mass"])

FileNotFoundError: [Errno 2] No such file or directory: 'data_Skim_GamGam.root'

# NOTE - NONE OF THE REST OF THIS NOTEBOOK WILL RUN UNTIL YOU DOWNLOAD THE INPUT DATA AND SIMULATION FILES FROM HERE. https://drive.google.com/drive/folders/1iqUqc277CDBXsfUks0Z3i9EzhH0C5_Fp?usp=sharing

 - You will need to be logged in to google with your UCT account to gain access
 - PLACE ALL FILES IN THE SAME DIRECTORY AS THIS NOTEBOOK.

# Step 2. Apply a series of event selections
    - only select events containing two photons with specific pt and \eta

In [ ]:
cuts0 = df[(df["photon_pt[0]"] > 35000)  & (df["photon_pt[1]"] > 25000 )]

cuts1 = cuts0[( np.abs(cuts0["photon_eta[0]"]) < 2.37)  & ( np.abs(cuts0["photon_eta[1]"]) < 2.37 )]

#alternative eta cut, selects central photons only
#cuts1 = cuts0[( np.abs(cuts0["photon_eta[0]"]) < 0.75)  & ( np.abs(cuts0["photon_eta[1]"]) < 0.75 )]

#these cuts exclude a small region of the ATLAS detector known to have poor efficiency and resolution.
cuts1a = cuts1[ (np.abs(cuts1["photon_eta[0]"]) < 1.37) | (np.abs(cuts1["photon_eta[0]"]) > 1.52) ]
cuts1b = cuts1a[ (np.abs(cuts1a["photon_eta[1]"]) < 1.37) | (np.abs(cuts1a["photon_eta[1]"]) > 1.52) ]


cuts2a = cuts1b[(cuts1b["photon_pt[0]"] / cuts1b["diphoton_mass"])  > 0.35]
cuts2b = cuts2a[(cuts2a["photon_pt[1]"] / cuts2a["diphoton_mass"])  > 0.25]

cuts3 = cuts2b[(cuts2b["diphoton_mass"]  > 105000) & (cuts2b["diphoton_mass"]  < 160000) ]

finalData =  cuts3

print("Number of selected events = " + str(len(finalData.index)))

finalData

# Step 3. Combine the four vectors of the photons

In [ ]:
flatData0 = LVepm.TLorentzVectorArray.from_ptetaphi(finalData["photon_pt[0]"], finalData["photon_eta[0]"], finalData["photon_phi[0]"],finalData["photon_E[0]"])
flatData1 = LVepm.TLorentzVectorArray.from_ptetaphi(finalData["photon_pt[1]"], finalData["photon_eta[1]"], finalData["photon_phi[1]"],finalData["photon_E[1]"])

flatData = flatData0 + flatData1

# Step 4. Create a histogram of the masses of the diphoton system.

In [ ]:
mH = 124894.0
sigmaH = 210.0

#example binning
nBins = 30
minMass = 105000
maxMass = 160000

countsData, edges = np.histogram(flatData.mass, bins=nBins, range=(minMass,maxMass))

#create an array of the centre of each bin, useful for plotting the pdf later
centres = (edges[1:] + edges[:-1]) / 2

# Step 5. Fit a background model to data and make a comparison plot.

- create a statistical model for the diphoton mass distribution composed of of a gaussian ('norm') pdf for the $H\rightarrow \gamma \gamma$ process and a third-order polynomial fuction for the background diphoton processes. The mean of the signal gauusian distribution is chosen as the measured $m_{H}$ value.

In [ ]:
b = np.poly1d(np.polyfit(centres, countsData, 3))

#guess a signal integral for illustration purposes
integral = 10113802.23

s = ( scipy.stats.norm.pdf(centres, mH, sigmaH) * integral)
sb = s+b(centres)

plt.figure()
plt.xlabel("m phph")
plt.ylabel("events per bin")
plt.plot(centres, s, 'b', linewidth=1, label="Potential Signal")
plt.plot(centres, b(centres), 'k', linewidth=1, linestyle="--", label="Background")
plt.plot(centres, sb, 'r', linewidth=2, label="Signal+Background")

plt.errorbar(centres, countsData, yerr=np.sqrt(countsData), fmt='o', mfc='k', mec='k',ms=4, mew=0.2, ecolor='k',label="ATLAS Open Data",)
plt.legend()

# Step 6. Create a plot of the binned "residuals" vs $m_{H}$

- Subtract the polynomial background prediction from the data to calculate the \emph{residuals}. 
- The residual in each bin is our estimate of the signal in that bin.

In [ ]:
%%time
#ratio plot 
residuals = countsData - b(centres) 
plt.figure()
plt.xlabel("$ m_{\gamma\gamma}$")
plt.ylabel("Data - background")
plt.errorbar(centres, residuals, yerr=np.sqrt(countsData), label="Estimated Signal (Data - background)", fmt='o', mfc='k', mec='k', mew=0.2, ecolor='k')

nSteps = 100

plt.plot(centres, s, 'b', linewidth=1, label="Potential signal")
plt.plot(centres, np.zeros(len(centres)), 'k', linewidth=1, linestyle="--", label="Background")
plt.legend()
plt.show()

# Step 7.  Simultaneously fit  $m_{H}$, $\Gamma_{H}$ and $N_{H\rightarrow\gamma\gamma}$ from the residuals
- If we assume the signal will have a gaussian shape with $\mu \approx m_{H} \approx$ 125000 MeV, we can use python libraries to fit a gaussian distrubution to the residual graph and estimate the number of signal events $N_{H\rightarrow\gamma\gamma}$ in the data as well as $m_{H}$ and $\Gamma_{H}$.

In [ ]:
#assume mass, sigma
from scipy.optimize import curve_fit

def gaussian(x, mean, amplitude, standard_deviation):
    return amplitude * np.exp( - ((x - mean) / standard_deviation) ** 2)

bestFitParams, covarianceMatrix = curve_fit(gaussian, centres, residuals, p0=[125000., 200., 1000.])

#the p0 values are that the 'starting points' for 𝑚𝐻, Γ𝐻 and 𝑁𝐻→𝛾𝛾 in the fit.

x_interval_for_fit = np.linspace(edges[0], edges[-1], 10000)
plt.plot(x_interval_for_fit, gaussian(x_interval_for_fit, *bestFitParams), label='$ H \gamma \gamma$ signal')
plt.errorbar(centres, residuals, yerr=np.sqrt(countsData), label="Data - Background", fmt='o', mfc='k', mec='k', mew=0.2, ecolor='k')
plt.legend()
plt.xlabel("$ m_{\gamma\gamma}$")
plt.ylabel("Data - prediction")

# add the values of thr fitted signal shape at a series of points to estimate the total 
# number of signal events
limA = 11500
limA = 13500
sigIntegral = 0.0
stepSize = (maxMass - minMass)/(nSteps)

xStep = minMass

for p in range(0, nSteps):
        xStep = xStep + stepSize
        sigHeight = gaussian(xStep, *bestFitParams)
        #print(sigHeight)
        sigIntegral = (sigIntegral + (sigHeight) )

print("best fitting parameters:")
print("MH = " + str(float(bestFitParams[0])))
print("sigmaH = " + str(float(bestFitParams[1]) ))
print("NH = " + str(float(sigIntegral) ))

# Step 8  Estimate the statistical significance of the $H\rightarrow \gamma \gamma$ signal.




In [ ]:
#Your code goes here!

# Step 9 - Your turn!

It's time for you to try improvements to the analysis especially if they can improve the significance of the signal.
Refer to the lab manual for suggested modifications.

